<a href="https://colab.research.google.com/github/Amruth-varsh/Amruth-varsh/blob/main/Amruthvarsh_infy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os

# Function to convert video to frames
def video_to_frames(video_path, output_folder, max_frames=100):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Capture the video
    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as an image file
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

    # Release the video capture object
    cap.release()
    print(f"Extracted {frame_count} frames.")

# Example usage
video_path = '/content/Input Videoo.mp4'
output_folder = '/content/Frames'                 # Output folder path
video_to_frames(video_path, output_folder)

Extracted 100 frames.


In [ ]:
import os
import json
import pandas as pd
from PIL import Image

# Path to the uploaded CSV file
csv_path = '/content/Player and ball.csv'
# Directory where your images are stored (upload your images to this directory)
images_dir = '/content/Frames'  # Modify this to the path where your images are located
# Directory to save YOLOv5 labels
labels_dir = '/content/labels/'

# Create the labels directory if it doesn't exist
os.makedirs(labels_dir, exist_ok=True)

# Load the CSV file
df = pd.read_csv(csv_path)

# List of filenames for the 100 specific frames
# You should replace the below list with the actual filenames from your frameset
specific_frames = [
    'frame_001.jpg', 'frame_002.jpg', 'frame_003.jpg',  # and so on for your 100 frames
    # ...
]

# Function to convert a circle to YOLO format (x_center, y_center, width, height)
def circle_to_yolo(cx, cy, r, img_w, img_h):
    x_center = cx / img_w
    y_center = cy / img_h
    width = (2 * r) / img_w
    height = (2 * r) / img_h
    return x_center, y_center, width, height

# Loop over the dataframe and generate the label files for specific frames
for _, row in df.iterrows():
    # Extract filename and region shape attributes (assuming circle shape)
    filename = row['filename']
    if filename in specific_frames:  # Process only if the filename is in the specific frames list
        region_shape = json.loads(row['region_shape_attributes'])

        # Only process if the region shape is a circle
        if region_shape['name'] == 'circle':
            cx = region_shape['cx']
            cy = region_shape['cy']
            r = region_shape['r']

            # Load the corresponding image to get its dimensions
            image_path = os.path.join(images_dir, filename)
            if os.path.exists(image_path):
                # Open the image to get its width and height
                img = Image.open(image_path)
                img_w, img_h = img.size

                # Convert circle to YOLO format
                x_center, y_center, width, height = circle_to_yolo(cx, cy, r, img_w, img_h)

                # Save the annotation in YOLOv5 format (class_id 0 for 'ball')
                label_file = os.path.join(labels_dir, os.path.splitext(filename)[0] + '.txt')
                with open(label_file, 'a') as f:
                    f.write(f'0 {x_center} {y_center} {width} {height}\n')

print("YOLO annotations for specific frames generated successfully.")


YOLO annotations for specific frames generated successfully.


In [ ]:
import os
import shutil
import pandas as pd
import random
from PIL import Image

# Define paths
images_folder = '/content/Frames'  # Update to your folder containing images
annotations_csv = '/content/Player and ball.csv'  # Path to your CSV file from VGG Annotator
output_folder = '/content/dataset'  # Folder to save the dataset

# Create folders for YOLO dataset structure
train_images_folder = os.path.join(output_folder, 'images', 'train')
val_images_folder = os.path.join(output_folder, 'images', 'val')
train_labels_folder = os.path.join(output_folder, 'labels', 'train')
val_labels_folder = os.path.join(output_folder, 'labels', 'val')

# Create directories if they don't exist
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)

# Load annotations from CSV
annotations = pd.read_csv(annotations_csv)

# Prepare label files
label_files = {}
for _, row in annotations.iterrows():
    img_name = row['filename']  # Ensure this column matches your CSV's filename column
    img_path = os.path.join(images_folder, img_name)

    if os.path.exists(img_path):
        # Example: Extract bounding box from VGG Annotator's JSON-like structure
        region_shape = eval(row['region_shape_attributes'])  # Replace this with the correct way to access the attributes
        # Assuming region_shape has keys: 'x', 'y', 'width', 'height'
        xmin = region_shape['x']
        ymin = region_shape['y']
        width = region_shape['width']
        height = region_shape['height']
        xmax = xmin + width
        ymax = ymin + height

        # Open the image to get its dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width_yolo = width / img_width
        height_yolo = height / img_height

        if img_name not in label_files:
            label_files[img_name] = []

        # Assuming class_id for your object is 0 (update as needed)
        label_files[img_name].append(f"0 {x_center} {y_center} {width_yolo} {height_yolo}")

# Write YOLO label files
for img_name, annotations in label_files.items():
    label_file_path = os.path.join(train_labels_folder if random.random() < 0.8 else val_labels_folder, f"{os.path.splitext(img_name)[0]}.txt")
    with open(label_file_path, 'w') as label_file:
        label_file.write("\n".join(annotations))

# Now copy the images to the appropriate folders
for img_name in label_files.keys():
    img_src_path = os.path.join(images_folder, img_name)
    img_dest_path = os.path.join(train_images_folder if random.random() < 0.8 else val_images_folder, img_name)
    shutil.copy(img_src_path, img_dest_path)

print("YOLOv5 dataset preparation from VGG Annotator CSV complete!")

#print("YOLOv5 dataset preparation complete!")


YOLOv5 dataset preparation from VGG Annotator CSV complete!


In [ ]:
import yaml

# Define the paths and classes
data_yaml = {
    'train': '/content/dataset/images/train',       # Training images folder
    'val': '/content/dataset/images/val',           # Validation images folder
    'nc': 14,                      # Number of classes
    'names': ['1', '2', '3','4','5','6','7','8','9','10','11','12','13','14']              # Replace with your actual class names
}

# Save the YAML file
with open('/content/dataset.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print("dataset.yaml file created successfully!")


dataset.yaml file created successfully!


In [ ]:
!git clone https://github.com/ultralytics/yolov5


Cloning into 'yolov5'...
remote: Enumerating objects: 17045, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17045 (delta 10), reused 18 (delta 8), pack-reused 17022 (from 1)
Receiving objects: 100% (17045/17045), 15.66 MiB | 14.57 MiB/s, done.
Resolving deltas: 100% (11700/11700), done.


In [ ]:
%cd yolov5
!pip install -r requirements.txt

/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.3/886.3 kB 18.1 MB/s eta 0:00:00


In [ ]:
!python train.py --img 640 --batch 8 --epochs 50 --data /content/dataset.yaml --weights yolov5s.pt --name court_model


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/court_model2/weights/best.pt --source /content/Input\ Videoo.mp4 --project /content/Outputvideo --imgsz 640 --conf-thres 0.25 --name Outputvideo


python3: can't open file '/content/detect.py': [Errno 2] No such file or directory


In [ ]:
import shutil

# Create a zip file of the Outputvideo directory
shutil.make_archive('/content/Outputvideo', 'zip', '/content/Outputvideo')

# Provide a download link
from google.colab import files
files.download('/content/Outputvideo.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Replace the path with the correct path to your best.pt file
file_path = '/content/yolov5/runs/train/court_model2/weights/best.pt'

# Download the file
files.download(file_path)


FileNotFoundError: Cannot find file: /content/yolov5/runs/train/court_model2/weights/best.pt

In [ ]:
import os

# Replace with the directory you want to check
path_to_check = '/content/yolov5/runs/train/court_model2/weights/'

# List files in the directory
if os.path.exists(path_to_check):
    print("Files in directory:", os.listdir(path_to_check))
else:
    print("Directory does not exist:", path_to_check)


Directory does not exist: /content/yolov5/runs/train/court_model2/weights/


In [ ]:
import cv2
import pandas as pd
import ast
import torch  # Ensure PyTorch is imported for YOLOv5

# Paths
video_path = '/content/Input Videoo.mp4'
csv_path = '/content/Outputvideo/Outputvideo/court_points_csv.csv'
output_video_path = '/content/court_output_video.mp4'
weights_path = '/content/yolov5/runs/train/court_model2/weights/best.pt'

# Load court points from CSV
court_points = pd.read_csv(csv_path)

# Initialize video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codec and create VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)

# Track the number of annotated frames processed
processed_frames = 0
total_annotated_frames = 100  # Limit processing to 100 frames

# Process video frame by frame
while processed_frames < total_annotated_frames:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
    frame_filename = f'frame_{str(current_frame).zfill(4)}.jpg'

    # Perform object detection on the frame
    results = model(frame)  # Detect players and balls

    # Get results as a DataFrame
    detections = results.pandas().xyxy[0]  # Get detection results
    # Filter detections for players and balls
    for index, row in detections.iterrows():
        if row['name'] in ['player', 'ball']:  # Replace with actual class names
            x1, y1, x2, y2, conf, cls = row['xmin'], row['ymin'], row['xmax'], row['ymax'], row['confidence'], row['name']
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box
            cv2.putText(frame, f"{cls} {conf:.2f}", (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Filter court points for the current frame
    frame_points = court_points[court_points['filename'] == frame_filename]

    if not frame_points.empty:
        print(f"Processing frame {current_frame}: Overlaying court points from {frame_filename}")

        # Draw each court point on the frame
        for _, row in frame_points.iterrows():
            # Parse 'region_shape_attributes' to get cx and cy
            attributes = ast.literal_eval(row['region_shape_attributes'])
            x, y = int(attributes['cx']), int(attributes['cy'])

            # Draw a larger circle with a distinct color (e.g., red) for visibility
            cv2.circle(frame, (x, y), radius=8, color=(0, 0, 255), thickness=-1)

            # Label each point with "Court"
            cv2.putText(frame, "Court", (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 0, 255), 2, cv2.LINE_AA)

        processed_frames += 1

    # Write the frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved at: {output_video_path} with {processed_frames} frames processed.")


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-11-5 Python-3.10.12 torch-2.5.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7047883 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarnin

Processing frame 0: Overlaying court points from frame_0000.jpg
Processing frame 1: Overlaying court points from frame_0001.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processing frame 2: Overlaying court points from frame_0002.jpg
Processing frame 3: Overlaying court points from frame_0003.jpg
Processing frame 4: Overlaying court points from frame_0004.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processing frame 95: Overlaying court points from frame_0095.jpg
Processing frame 96: Overlaying court points from frame_0096.jpg
Processing frame 97: Overlaying court points from frame_0097.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processing frame 98: Overlaying court points from frame_0098.jpg
Processing frame 99: Overlaying court points from frame_0099.jpg
Output video saved at: /content/court_output_video.mp4 with 100 frames processed.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
import shutil

# Path to the output video
output_video_path = '/content/court_output_video.mp4'

# Create a zip file of the output video
shutil.make_archive('/content/court_output_video', 'zip', '/content', 'court_output_video.mp4')

# Provide a download link for the zip file
from google.colab import files
files.download('/content/court_output_video.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import pandas as pd
import ast
import torch

# Paths
video_path = '/content/Input Videoo.mp4'
csv_path = '/content/court_points_csv.csv'
output_video_path = '/content/court_output_video.mp4'
weights_path = '/content/yolov5/runs/train/court_model2/weights/best.pt'

# Load court points from CSV
court_points = pd.read_csv(csv_path)

# Initialize video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codec and create VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

# Track the number of annotated frames processed
processed_frames = 0
total_annotated_frames = 100  # Limit processing to 100 frames

# Process video frame by frame
while processed_frames < total_annotated_frames:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
    frame_filename = f'frame_{str(current_frame).zfill(4)}.jpg'

    # Perform object detection on the frame
    results = model(frame)  # Detect players and balls

    # Get results as a DataFrame
    detections = results.pandas().xyxy[0]  # Get detection results
    # Filter detections for players and balls
    for index, row in detections.iterrows():
        if row['name'] in ['player', 'ball']:  # Replace with actual class names
            x1, y1, x2, y2, conf, cls = row['xmin'], row['ymin'], row['xmax'], row['ymax'], row['confidence'], row['name']
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box
            cv2.putText(frame, f"{cls} {conf:.2f}", (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Filter court points for the current frame
    frame_points = court_points[court_points['filename'] == frame_filename]

    if not frame_points.empty:
        print(f"Processing frame {current_frame}: Overlaying court points from {frame_filename}")

        # Draw each court point on the frame
        for _, row in frame_points.iterrows():
            # Parse 'region_shape_attributes' to get cx and cy
            attributes = ast.literal_eval(row['region_shape_attributes'])
            x, y = int(attributes['cx']), int(attributes['cy'])

            # Draw a larger circle with a distinct color (e.g., red) for visibility
            cv2.circle(frame, (x, y), radius=8, color=(0, 0, 255), thickness=-1)

            # Label each point with "Court"
            cv2.putText(frame, "Court", (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 0, 255), 2, cv2.LINE_AA)

        processed_frames += 1

    # Write the frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved at: {output_video_path} with {processed_frames} frames processed.")
